# Import Packages

In [119]:
import sys
 
import os
import pandas as pd
import numpy as np
import pickle
import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt

import Lib_RWA as Lib_RWA

# delete lib_RWA from system modules 

del(sys.modules['Lib_RWA'])

# delete SFR_PD_Recalibration_2023_Lib from system modules

del(sys.modules['SFR_PD_Recalibration_2023_Lib'])


import SFR_PD_Recalibration_2023_Lib as tool

import Lib_RWA as Lib_RWA

import math

In [120]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [121]:

# Snaphsot and RWA file information

# snapshot_num = 202212 # 20250403 Fran commented out as George in S1
# snapshot = str(snapshot_num) # 20250403 Fran commented out as George in S1
# snapshot_date = '2022-12-31' # 20250403 Fran commented out as George in S1
# RWA_File_from_Chenxi = 'SFR_RWA_202212.csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

snapshot_num = 202412 # 20250403 Fran added as George in S1 
snapshot = str(snapshot_num) # 20250403 Fran added as George in S1
snapshot_date = '2024-12-31' # 20250403 Fran added as George in S1
RWA_File_from_Chenxi = 'SFR_202412_v2 (from Chenxi 20250403).csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

In [122]:
#CAR 2023 Ch5 prescribed parameters

correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023

In [ ]:
# Columns to Keep

RWA_Info_Cols_to_Keep = ['Loan_Number','Insured_class','EAD','Advance_Amount','Years_to_maturity', 'corr_uninsured', 'rwa_airb','RWA_standardized']

PD_Cols_to_Keep = ['Loan_Number','SL_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin','Insured_Ind','Alt_Prime_Indicator','RemainingPrincipal_Tot_Exp','RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind','Combo_Province_Metro_Override_WOE','Combo_LTV_Insured_Ind']

LGD_Cols_to_Keep = ['Loan_Number','MRS_Bin','LR_Avg_LGD','Segment_Avg_LGD','Base_Line_LGD','LGD_DT_Adjusted','Final_LGD', 'Occupancy']






# Set Folder

In [123]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added


if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added




# Load RWA Data Info

In [ ]:

# Read in the RWA data from Chenxi

# Import data with Loan_Number,Insured_class,CalibratedPD,Final_LGD,EAD,Advance_Amount,Years_to_maturity and corr_uninsured
##################### load snapshot data Chenxi provided and rename#################################
#rwa_data = pd.read_csv(input_dir +'\\'+'SFR_RWA_' + snapshot + '.csv', low_memory=False) #20250403 Fran commented as George in S1
rwa_data = pd.read_csv(input_dir +'\\'+ RWA_File_from_Chenxi, low_memory=False) #20250403 Fran added as George in S1
#rename columns RWA_AIRB to rwa_airb

rwa_data.rename(columns={'RWA_AIRB':'rwa_airb'}, inplace=True)
# rwa_raw_data_0 = rwa_data[['Loan_Number','Insured_class','CalibratedPD','Final_LGD','EAD','Advance_Amount','Years_to_maturity', 'corr_uninsured', 'rwa_airb','insured_PMI_ratio','corr_insured','Maturity_adj','RWA_standardized']].rename({'CalibratedPD': 'CalibratedPD_old','Final_LGD': 'Final_LGD_old','insured_PMI_ratio':'insured_PMI_ratio_old','corr_insured':'corr_insured_old','Maturity_adj':'Maturity_adj_old'}, axis='columns')   # 20250410 George commented out as we don't need to rename the columns

rwa_raw_data_0 = rwa_data[RWA_Info_Cols_to_Keep]  # 20250410 George add

In [125]:
rwa_data.shape

(53402, 17)

In [126]:
#Floor the EAD on 0          

rwa_raw_data_0.loc[rwa_raw_data_0['EAD']<0, ['EAD']] =0


# Load PD Data

In [127]:
# add new SFR PD model generated PD 


#new_pd_data = pd.read_csv( Active_dir +'\\'+'PARAM_Pred_PD'  + '.csv', low_memory=False) # 20250402 George Commented out 
new_pd_data = pd.read_pickle(input_dir + '\\Rating_July2020_to_Dec2024_EQB (from George 20250402).pkl') # 20250402 George added
new_pd_data_red = new_pd_data.loc[new_pd_data['SL_Date'] == snapshot_date]

# For columns PD_Post_MOC_Pre_Adj and PD_Pre_MOC, if not in the data set, add them with nan #20250402 George added

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added

if 'PD_Pre_MOC' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Pre_MOC'] = np.nan #20250402 George added


# Rename AIRB_PD_Risk_Rating to be MRS_Bin #20250402 George added

new_pd_data_red = new_pd_data_red.rename({'AIRB_PD_Risk_Rating':'MRS_Bin'}, axis='columns')  #20250402 George added

C:\Users\georgez\AppData\Local\Temp\ipykernel_12632\1811959286.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added


In [128]:
# Save Loan number list for Chenxi as she requested to check
with pd.ExcelWriter(Output_dir +'\\'+'eqb_loans_list_dec24.xlsx' )as writer:
    new_pd_data_red['Loan_Number'].to_excel(writer, sheet_name='loan_list', index=False)

In [129]:
# Remove Dafault Loans

# print columns in alphabetical order in the new_pd_data_red dataframe #20250402 George added

print(sorted(new_pd_data_red.columns)) #20250402 George added

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Before Removing Default Loans*****************') #20250402 George added
print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

# Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['PD_Post_MOC'] > 0) & (new_pd_data_red['PD_Post_MOC'] < 1)] #20250402 George added

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

print('***************After Removing Default Loans*****************') #20250402 George added

print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

['Alt_Prime_Indicator', 'AppraisalValue_Inception', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Funded_Date', 'Insured_Ind', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Metro_Region_BF_FMT', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'SL_Date', 'Sub_Product', 'log_odds']
***************Before Removing Default Loans*****************
  MRS_Bin Loan_Number PD_Post_MOC          
              nunique         min       max
0     1.0       26705    0.002228  0.002228
1     2.0       14383    0.003622  0.003622
2     3.0        5505    0.007377  0.007377
3     4.0    

## Check Corporate Loan

In [130]:
# Group by SL_Date and Corporate_Applicant_Ind, count number of Loan_Number

print('***************Group by SL_Date and Corporate_Applicant_Ind*****************') #20250403 George added

print(new_pd_data_red.groupby(['SL_Date','Corporate_Applicant_Ind'], as_index=False).agg({'Loan_Number': pd.Series.nunique})) #20250403 George added

***************Group by SL_Date and Corporate_Applicant_Ind*****************
     SL_Date  Corporate_Applicant_Ind  Loan_Number
0 2024-12-31                      0.0        53837
1 2024-12-31                      1.0         1410


In [131]:
# Compare with RWA data from Chenxi

# Outer Join new_pd_data_red with rwa_raw_data_0 on Loan_Number #20250403 George added

rwa_raw_data_pd_check = pd.merge(rwa_raw_data_0['Loan_Number'], new_pd_data_red[['Loan_Number','Corporate_Applicant_Ind']], how='outer', on=['Loan_Number'], indicator= 'merge_') #20250403 George added

# change the content to be more meaningful #20250403 George added

rwa_raw_data_pd_check['merge_'] = rwa_raw_data_pd_check['merge_'].replace({'left_only': 'Loan in Chenxi only', 'right_only': 'Loan in PD Data only', 'both': 'Loan in both'}) #20250403 George added


# group by merge_ and Corporate_Applicant_Ind, including missing values, count number of Loan_Number #20250403 George added

print('***************Group by merge_ and Corporate_Applicant_Ind*****************') #20250403 George added

print(rwa_raw_data_pd_check.groupby(['merge_','Corporate_Applicant_Ind'], dropna= False).agg({'Loan_Number': pd.Series.nunique})) #20250403 George added

***************Group by merge_ and Corporate_Applicant_Ind*****************
                                              Loan_Number
merge_               Corporate_Applicant_Ind             
Loan in Chenxi only  NaN                            396.0
                     0.0                              NaN
                     1.0                              NaN
Loan in PD Data only NaN                              NaN
                     0.0                            834.0
                     1.0                           1407.0
Loan in both         NaN                              NaN
                     0.0                          53003.0
                     1.0                              3.0


# Load LGD Data

In [133]:
# add new SFR LGD model generated LGD and apply LGD floor 

# 2023_11-04 is most recent one from Ben
#new_lgd_data = pd.read_csv(input_dir+'\Dec_2022_LGD_2023_11_04_Recalibrated_Model.csv', low_memory = False)

if snapshot_num == 202212:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2022_12 (from Abhi 20250410).pkl') # 20250406 George added

if snapshot_num == 202412:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2024_12 (from Abhi 20250404).pkl') # 20250406 George added


In [134]:
new_lgd_data.head()

,Insured_Ind,Sub_Product,Default_Ind,LoanType,RemainingPrincipal_Excl_Partner,Loan_Number,SL_Date,Foreclosure_Ind,Province_Foreclosure,BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw,BF_Appr_Prov_Dw,Occupancy,LTV_Bin,Appraisal_Bin,LTV_Bin_WOE,Appraisal_Bin_WOE,LTV_Bin_WOE,Appraisal_Bin_WOE,Foreclosure_Ind_WOE,Province_Foreclosure_WOE,Occupancy_WOE,Pred_LGD,Final_LGD,Mapped_LGD,MRS_Bin,LR_Avg_LGD
45260,Insured,EQB,0,13,2879.34,78371,2024-12-31,0.0,ON,NaN,NaN,Owner_Occupied,NaN,NaN,-0.709019,-1.030904,-0.709019,-1.030904,0.476228,0.601066,0.062674,0.102871,0.5355,0.101136,4,0.110595
45379,Insured,EQB,0,13,62101.09,78970,2024-12-31,0.0,ON,NaN,NaN,Owner_Occupied,NaN,NaN,-0.709019,-1.030904,-0.709019,-1.030904,0.476228,0.601066,0.062674,0.102871,0.5355,0.101136,4,0.110595
45382,Insured,EQB,0,13,10786.51,81151,2024-12-31,0.0,ON,NaN,NaN,Owner_Occupied,NaN,NaN,-0.709019,-1.030904,-0.709019,-1.030904,0.476228,0.601066,0.062674,0.102871,0.5355,0.101136,4,0.110595
41631,Insured,EQB,0,13,48060.61,81246,2024-12-31,0.0,ON,NaN,NaN,Owner_Occupied,NaN,NaN,-0.709019,-1.030904,-0.709019,-1.030904,0.476228,0.601066,0.062674,0.102871,0.5355,0.101136,4,0.110595
45383,Not Insured,EQB,0,30,29783.19,81701,2024-12-31,0.0,ON,NaN,NaN,Rental,NaN,NaN,-0.709019,-1.030904,-0.709019,-1.030904,0.476228,0.601066,-0.450006,0.159290,0.5355,0.101136,4,0.110595


In [135]:
# Add columns LGD_DT_Adjusted, Segment_Avg_LGD and Base_Line_LGD if they are not in the data set #20250412 George added

if 'LGD_DT_Adjusted' not in new_lgd_data.columns: # 20250412 George added
    new_lgd_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

if 'Segment_Avg_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Segment_Avg_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

if 'Base_Line_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Base_Line_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

# Merge PD and LGD data for Corporate Loans

In [ ]:
# join new_Pd_data_red and new_lgd_data on Loan_Number

pd_data_red_for_Cor = new_pd_data_red.rename({'MRS_Bin':'MRS_Bin_PD'}, axis = 'columns') #20250402 George added
new_lgd_data_for_Corp = new_lgd_data.rename({'MRS_Bin':'MRS_Bin_LGD'}, axis = 'columns') #20250402 George added

PD_LGD_Merge = pd.merge(pd_data_red_for_Cor[['Loan_Number','MRS_Bin_PD','Corporate_Applicant_Ind','RemainingPrincipal_Tot_Exp','Insured_Ind','RemainingPrincipal_Excl_Partner']], new_lgd_data_for_Corp[['Loan_Number','MRS_Bin_LGD']], how='left', on=['Loan_Number'])



In [ ]:

# Filter with Corporate_Applicant_Ind = 1 #20250404 George added

PD_LGD_Merge_Corp = PD_LGD_Merge.loc[PD_LGD_Merge['Corporate_Applicant_Ind'] == 1] #20250404 George added

# Group by Insured_Ind, MRS_Bin_PD and MRS_Bin_LGD, count number of Loan_Number and sum of RemainginPrincipal_Tot_Exp#20250404 George added

print('***************Corporate Loans Only*****************') #20250404 George added

print(PD_LGD_Merge_Corp.groupby(['Insured_Ind','MRS_Bin_PD','MRS_Bin_LGD'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'RemainingPrincipal_Tot_Exp': sum})) #20250404 George added


***************Corporate Loans Only*****************
    Insured_Ind  MRS_Bin_PD  MRS_Bin_LGD  Loan_Number  RemainingPrincipal_Tot_Exp
0   Not Insured         1.0            1          225                9.839399e+07
1   Not Insured         1.0            2           91                4.079501e+07
2   Not Insured         1.0            3           15                7.153886e+06
3   Not Insured         1.0            4            1                2.092401e+05
4   Not Insured         2.0            1          110                6.913848e+07
5   Not Insured         2.0            2          268                1.580170e+08
6   Not Insured         2.0            3          175                1.119233e+08
7   Not Insured         2.0            4           69                3.055346e+07
8   Not Insured         3.0            1           31                2.121737e+07
9   Not Insured         3.0            2          102                6.558273e+07
10  Not Insured         3.0            3     

In [ ]:

# Save data to excel #20250404 George added

with pd.ExcelWriter(Output_dir +'\\'+'Corporate_Loans_Breakdown.xlsx' )as writer:

    PD_LGD_Merge_Corp.to_excel(writer, sheet_name='Corp_Loans', index=False) #20250404 George added

# Merge RWA Data Info with PD Data

In [ ]:
#test = new_pd_data_red[ ['PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC'] ].drop_duplicates()


rwa_raw_data_1 = pd.merge(rwa_raw_data_0, new_pd_data_red[PD_Cols_to_Keep], how='left',on=['Loan_Number'] )

rwa_raw_data_2 =  rwa_raw_data_1.loc[~rwa_raw_data_1['PD_Pre_MOC'].isnull()] #remove records not existing in Chenxi's data since they cannot be compared
rwa_raw_data_3 = rwa_raw_data_2.rename({'MRS_Bin':'MRS_Bin_PD'}, axis = 'columns')

# summary_table = rwa_raw_data_3[['CalibratedPD_old','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC']].describe()  # 20250411 George commented out as we don't need to create the summary table

# summary_table.to_excel(excel_writer=file_dir+'\Summary_RWA_'+snapshot+'.xlsx', sheet_name='Summary_table')

# Merge RWA info with LGD Data

In [ ]:

#Gen3 LGD data, only including uninsured loans

rwa_raw_data_4 = pd.merge(rwa_raw_data_3,new_lgd_data[LGD_Cols_to_Keep], how='left', on= ['Loan_Number'])
rwa_raw_data_4 = rwa_raw_data_4.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')



In [137]:

# Remove 30 loans due to data inconsistencies

#For Dec 2022 snapshot, all "insured" loans do not have new LGD attached; only 30 "uninsured" loans do not have new LGD. Proceed. Checked these 30 loans are insured ones in PD drivers and the inconsistencies is between Chenxi's vs. Surinder's. MD remove these 30 from RWA calculation  # Jason's old comment


rwa_raw_data_4 = rwa_raw_data_4.loc[ ~ ((rwa_raw_data_4['Model_LGD'].isna() )& ( rwa_raw_data_4['Insured_class']== 'Uninsured'))]
rwa_raw_data_4.shape
rwa_raw_data_4.head()

,Loan_Number,Insured_class,EAD,Advance_Amount,Years_to_maturity,corr_uninsured,rwa_airb,RWA_standardized,SL_Date,PD_Pre_MOC,PD_Post_MOC_Pre_Adj,PD_Post_MOC,MRS_Bin_PD,Insured_Ind,Alt_Prime_Indicator,RemainingPrincipal_Tot_Exp,RemainingPrincipal_Excl_Partner,Corporate_Applicant_Ind,Combo_Province_Metro_Override_WOE,Combo_LTV_Insured_Ind,MRS_Bin_LGD,LR_Avg_LGD,Segment_Avg_LGD,Base_Line_LGD,LGD_DT_Adjusted,Model_LGD,Occupancy
0,100031,CMHC,154095.29,185600.68,2.253251,0.15,0.290595,0.000,2024-12-31,0.002667,NaN,0.003622,2.0,Insured,Alt,152636.34,152636.34,0.0,-0.389936,"Insured, LTV<=0.40",2,0.033370,0.029987,0.029987,NaN,0.1821,Owner_Occupied
1,100103,CMHC,91082.35,178107.95,2.414784,0.15,0.101277,0.000,2024-12-31,0.001538,NaN,0.002228,1.0,Insured,Alt,89759.18,89759.18,0.0,0.165650,"Insured, LTV<=0.40",1,0.013818,0.010912,0.010912,NaN,0.1074,Owner_Occupied
2,100337,CMHC,196700.48,270155.62,1.749487,0.15,0.218717,0.000,2024-12-31,0.001538,NaN,0.002228,1.0,Insured,Alt,195097.17,195097.17,0.0,0.165650,"Insured, LTV<=0.40",1,0.013818,0.010912,0.010912,NaN,0.1074,Rental
3,100360,Uninsured,71475.09,149738.22,1.809719,0.15,13288.668913,14295.018,2024-12-31,0.005465,NaN,0.007377,3.0,Not Insured,Alt,70279.33,70279.33,0.0,-0.389936,"Uninsured, LTV<=0.30",2,0.033370,0.029987,0.029987,NaN,0.1821,Owner_Occupied
4,100392,CMHC,112436.71,153218.68,1.749487,0.15,0.623504,0.000,2024-12-31,0.001538,NaN,0.002228,1.0,Insured,Alt,111246.89,111246.89,0.0,-0.389936,"Insured, 0.40<LTV<=0.60",2,0.033370,0.029987,0.029987,NaN,0.1821,Owner_Occupied


# Calculate RWA with 0 Addon

In [141]:
# Force Addon to be 0

rwa_raw_data_0Addon = rwa_raw_data_4.copy()
rwa_raw_data_0Addon['AddOn'] =0

In [142]:
print(CMHC_pd)

0.0001


In [143]:
# Run RWA calculation function

[df_out_0Addon, rwa_by_Insured_class_0Addon, rwa_by_MRS_Bin_0Addon, res_data_0Addon]  = Lib_RWA.rwa_calculation(df_input_data = rwa_raw_data_0Addon, lgd_gen_floor = lgd_gen_floor, CMHC_lgd = CMHC_lgd, CMHC_pd = CMHC_pd )



0
1
2


In [144]:
# Save Data

with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_addon_zero.xlsx' )as writer:
    res_data_0Addon.to_excel(writer, sheet_name='RWA_data', index=False)
    df_out_0Addon.to_excel(writer, sheet_name="RWA") 
    rwa_by_Insured_class_0Addon.to_excel(writer, sheet_name="rwa_by_Insured_class")
    rwa_by_MRS_Bin_0Addon.to_excel(writer, sheet_name="rwa_by_MRS_Bin")

# Get LGD Addon and Calculate RWA

In [145]:
## retrieve DLGD floor readily available AddOn from Finance Production database
# Get max Feedid
server_2 = 'EQDWP01'  
database_2 = 'ET_Finance_Production'   



def sql_max_FeedID( snapshot ):
    
    table = 'ET_Finance_Production.dbo.tb_RE_log' # 20250410 George added

    if pd.to_datetime(snapshot) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT max([FeedID]) as max_FeedID
       FROM {table}
       where Reporting_date =  {snapshot} 
             
            '''  
    return sql

Max_Feed_ID= tool.download_from_sql( server_2, database_2, sql_max_FeedID( "'" + snapshot_date + "'" ) )['max_FeedID'][0]

def sql_LGD_addon( Max_Feed_ID ):
    
    table = 'ET_Finance_Production.dbo.tb_X_WoE_LGD_Result' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
       [LoanNumber]
       ,[Add_on_LGD] as AddOn
       FROM {table}
       where RunID =  {Max_Feed_ID} 
            '''  
    return sql

Add_on_LGD= tool.download_from_sql( server_2, database_2, sql_LGD_addon(Max_Feed_ID) )

Add_on_LGD['LoanNumber'] = Add_on_LGD['LoanNumber'].astype(str).astype(int)

rwa_raw_data_TrueAddon = pd.merge(rwa_raw_data_4, Add_on_LGD, how='left', left_on= ['Loan_Number'],right_on=['LoanNumber'])


#rwa_raw_data_5['Final_LGD'] = rwa_raw_data_5['Final_LGD_old']
#rwa_raw_data_5['PD_Post_MOC'] = rwa_raw_data_5['CalibratedPD_old']

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)
c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


In [146]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_TrueAddon, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


In [147]:

# #################################################################
# ####################### RWA Caculation #########################
# #################################################################

# df_out=pd.DataFrame()

# # record result data 
# res_data=pd.DataFrame()


# for ind_contol in range(0,3):
#     print(ind_contol)

#     # baseline LGD, Baseline PD
#     if ind_contol==0:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
#     #  PD_Post_MOC, LGD_DT_JUST     
#     if ind_contol==1:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD', 'LGD_DT_Adjusted': 'Pre_final_LGD' }, axis='columns'); 
#     #  PD_Post_MOC, Final LGD     
#     if ind_contol==2:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD' , 'Model_LGD': 'Pre_final_LGD' }, axis='columns'); 
        
#     #  aseline LGD,  PD after long-run adjustment
#     '''
#     if ind_contol==3:
#         rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
        
#         adj_Longrun = pd.read_excel(os.path.join(adhoc_path, 'MOC_Summary.xlsx'), sheet_name= 'Sheet1') [['Adj_Longrun']].values[0].item()
#         rwa_raw_data['CalibratedPD'] =  rwa_raw_data['CalibratedPD']*(1+adj_Longrun)
#     '''
    
#     # Apply regulatory floors and derive the final LGD
#     rwa_raw_data['DLGD_floor'] = rwa_raw_data['Segment_Avg_LGD'] + rwa_raw_data['AddOn']
    
#     # sum(~rwa_raw_data['DLGD_floor'].isna())
#     # sum((~rwa_raw_data['Segment_Avg_LGD'].isna()) & (rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"])))
    
#     # sum(~rwa_raw_data['Pre_final_LGD'].isna())
#     # sum(~rwa_raw_data['Segment_Avg_LGD'].isna() )
#     # sum(~(rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]) ))

#     rwa_raw_data.loc[~rwa_raw_data['Pre_final_LGD'].isna(),'OSFI_LGD_floor'] = lgd_gen_floor
#     rwa_raw_data['Final_LGD'] = rwa_raw_data[['Pre_final_LGD','DLGD_floor','OSFI_LGD_floor']].max(axis='columns')

#     # Add LGD_Before_Floors to be Pre_final_LGD

#     rwa_raw_data['LGD_Before_Floors'] = rwa_raw_data['Pre_final_LGD']

#     # Add a column Model_LGD_Modified to be 1 if Final_LGD > Pre_final_LGD, else 0

#     rwa_raw_data['Model_LGD_Modified'] = rwa_raw_data[['Final_LGD','Pre_final_LGD']].apply(lambda x: 1 if x[0] > x[1] else 0, axis=1)  # 20250408 George added

#     # Add a column EAD_DLGD_Modified=EAD*Model_LGD_Modified, as requested by Joseph 20250408 George added

#     rwa_raw_data['EAD_DLGD_Modified'] = rwa_raw_data[['EAD','Model_LGD_Modified']].apply(lambda x: x[0]*x[1], axis=1)  # 20250408 George added

#     # sum(~rwa_raw_data['Final_LGD'].isna())
    
#     #for insured loans which do not have Gen3 LGD, previous Final LGD is used
#     #rwa_raw_data.loc[rwa_raw_data['Pre_final_LGD'].isna(),'Final_LGD'] = rwa_raw_data['Final_LGD_old']
    
    
#     ## CG and Sagen use 11% LGD 
#     rwa_raw_data.loc[rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]), 'Final_LGD' ] = CMHC_lgd

#     # sum(rwa_raw_data['Final_LGD'].isna())
    
    
#     ## insured_PMI_ratio 
#     #     
#     rwa_raw_data["deductible_amount"] = 0.1 * rwa_raw_data.Advance_Amount
    
#     rwa_raw_data["insured_PMI_ratio"] = np.where(rwa_raw_data.EAD > 0, (rwa_raw_data.EAD - rwa_raw_data.deductible_amount) / rwa_raw_data.EAD, 0)
    
#     # !!!! can be negative; When insured_PMI_ratio is negative it will be handle in cond_PMI_dec case.
    
    
#   #  sum( round(rwa_raw_data["insured_PMI_ratio"]*10**5) == round(rwa_raw_data["insured_PMI_ratio_old"] *10**5) ) #56478
    
#     ## Generate indicators 
#     cond_CMHC = (rwa_raw_data.Insured_class.isin(['CMHC']))
    
#     cond_PMI = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD > 0.1 * rwa_raw_data.Advance_Amount)

#     cond_PMI_dec = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD <= 0.1 * rwa_raw_data.Advance_Amount)
    
#     cond_UI = (rwa_raw_data.Insured_class.isin(["Uninsured"]))
    
#     ########################## Add a column EAD_Post_CRM 20250408 George added start #######################################

#     rwa_raw_data['EAD_Post_CRM'] = np.nan

#     # if cond_UI, then EAD_Post_CRM = EAD, 20250408 George added with discussion with Joseph

#     rwa_raw_data.loc[cond_UI, 'EAD_Post_CRM'] = rwa_raw_data.loc[cond_UI, 'EAD']

#     # if cond_CMHC, then EAD_Post_CRM = 0, 20250408 George added with discussion with Joseph

#     rwa_raw_data.loc[cond_CMHC, 'EAD_Post_CRM'] = 0

#     # cond_PMI, then EAD_Post_CRM =  deductible amount, 2024-04-08 George added with discussion with Joseph 

#     rwa_raw_data.loc[cond_PMI, 'EAD_Post_CRM'] = rwa_raw_data.loc[cond_PMI, 'deductible_amount']

#     # cond_PMI_dec, then EAD_Post_CRM = EAD, 20250408 George added with discussion with Joseph

#     rwa_raw_data.loc[cond_PMI_dec, 'EAD_Post_CRM'] = rwa_raw_data.loc[cond_PMI_dec, 'EAD']

#     ############################################ Add a column EAD_Post_CRM 20250408 George added start ######################################
    
#     ## Maturity adjustment
    
#     def func_maturity_adj(pd_value):
#         import numpy as np
#         adj = (0.11852 - 0.05478 * np.log(pd_value)) ** 2
#         return adj  ##CAR 2023 5.3.1(i) 66.
    
#     rwa_raw_data.loc[cond_UI | cond_PMI_dec, "Maturity_adj"] = rwa_raw_data.loc[:, ['CalibratedPD']].apply(lambda s: func_maturity_adj(s[0]), axis=1) #not actually used since retail RWA formula will be applied
    
#     rwa_raw_data.loc[cond_CMHC | cond_PMI, "Maturity_adj"] = func_maturity_adj(CMHC_pd)
    

#     ## 'corr_uninsured'  depends on rental income indicator and has two values correlation_residential_mortgages_rental,correlation_residential_mortgages
    
    
#     # correlation for CMHC and insured deductible for PMI ?
#     def correlation_corp(pd_value):
#         import numpy as np
#         #     corr =  round(0.12 * (1 - np.exp(-50 * pd_value)) / (1- np.exp(-50)) +
#         #                       0.24 * (1 - (1- np.exp(-50 * pd_value))/(1 - np.exp(-50))),6) ##CAR 2023 5.3.1(i) 66.
#         corr = 0.12 * (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)) + 0.24 * (
#                 1 - (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)))
#         return corr
    
#     rwa_raw_data.loc[cond_CMHC | cond_PMI, 'corr_insured'] = correlation_corp(CMHC_pd)
    
    
#     ## func_risk_weight_retail 
#     def func_risk_weight_retail(pd_value, lgd_value, corr):  # CAR 2023 Chapter 5 paragrah 81
#         import scipy.stats as st
#         pd_value = float(pd_value)
#         lgd_value = float(lgd_value)
#         corr = float(corr)
#         pd_UL = st.norm.cdf(
#             (1 - corr) ** (-0.5) * st.norm.ppf(pd_value) + (corr / (1 - corr)) ** (0.5) * st.norm.ppf(0.999))
#         K = pd_UL * lgd_value - pd_value * lgd_value
#         # return  round(pd_UL * lgd_value - pd_value * lgd_value,6)
#         return K
    
    
#     def func_risk_weight_corp(pd_value, lgd_value, corr, maturity_adj,
#                               effective_maturity):  # CAR 2023 Chapter 5 paragraph 66
#         import scipy.stats as st
#         a = (lgd_value * st.norm.cdf((1 - corr) ** (-0.5) * st.norm.ppf(pd_value) +
#                                      (corr / (1 - corr)) ** 0.5 * st.norm.ppf(0.999)))
#         tail_value = a - (pd_value * lgd_value)
#         adj = (1 + (effective_maturity - 2.5) * maturity_adj) / (1 - (1.5 * maturity_adj))
#         k = tail_value * adj
#         # return  round(tail_value * adj,6)
#         return k
    
    
#     # retail risk weight function for uninsured loans
#     rwa_raw_data['risk_weight_uninsured'] = 0
    
#     rwa_raw_data.loc[cond_UI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_UI, ['CalibratedPD', 'Final_LGD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], s[1], s[2]), axis=1)
    
    
#     # re-write uninsured deductible for PMI using borrower PD and 100% LGD and retail risk weight function
#     # df["LGD_uninsured"]=1.0
    
#     rwa_raw_data.loc[cond_PMI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI, ['CalibratedPD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
#     rwa_raw_data.loc[cond_PMI_dec, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI_dec, ['CalibratedPD', 'corr_uninsured']].apply(
#             lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
        
#     # insured CMHC including PMI insured backstop use non-retail risk function
#     rwa_raw_data['risk_weight_insured'] = 0
    
#     rwa_raw_data.loc[cond_CMHC, 'risk_weight_insured'] = rwa_raw_data.loc[cond_CMHC, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
#          lambda s: func_risk_weight_corp(CMHC_pd, CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
#     rwa_raw_data.loc[cond_PMI, 'risk_weight_insured'] = rwa_raw_data.loc[cond_PMI, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
#          lambda s: func_risk_weight_corp(CMHC_pd,  CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
    
#     ## RWA_res_mortgage 
#     def RWA_res_mortgage(capital_value, capital_value_insured, EAD_value, insured_ratio):
#         # insured_ratio: Ratio of insured
#         #     insured_RWA = round(capital_value_insured * 12.5 * EAD_value,0)
#         #     uninsured_RWA = round(capital_value * 12.5 * EAD_value,0)
#         insured_RWA = capital_value_insured * 12.5 * EAD_value
#         uninsured_RWA = capital_value * 12.5 * EAD_value
#         rwa_total = insured_ratio * insured_RWA + (1 - insured_ratio) * uninsured_RWA
#         return rwa_total
    
    
    
#     # 1. CMHC insured
#     rwa_raw_data.loc[cond_CMHC, 'RWA_new'] = rwa_raw_data.loc[cond_CMHC, [ 'risk_weight_insured', 'EAD']].apply(lambda s: RWA_res_mortgage(0, s[0], s[1], 1), axis=1)
    
    
      
#     # 2. Sagen and CG
#     rwa_raw_data.loc[cond_PMI, 'RWA_new'] = rwa_raw_data.loc[
#           cond_PMI, ['risk_weight_uninsured', 'risk_weight_insured', 'EAD', "insured_PMI_ratio"]].apply(lambda s: RWA_res_mortgage(s[0], s[1], s[2], s[3]), axis=1)

#     rwa_raw_data.loc[cond_PMI_dec, 'RWA_new'] = rwa_raw_data.loc[cond_PMI_dec, ['risk_weight_uninsured', 'EAD']].apply(
#           lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
#     # 3. Uninsured Loans
#     rwa_raw_data.loc[cond_UI, 'RWA_new'] = rwa_raw_data.loc[cond_UI, ['risk_weight_uninsured', 'EAD']].apply(
#           lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
#     #test = rwa_raw_data.loc[ (round(rwa_raw_data["rwa_airb"]*10**6) != round(rwa_raw_data["RWA_new"]*10**6) )  &  (rwa_raw_data["Insured_class"] == 'CMHC') ][['Insured_class',"rwa_airb",'RWA_new']]
    
#     #rwa_raw_data.loc[rwa_raw_data['Insured_class'] =='Uninsufred', 'RWA_new'].sum()
#     ## Summary table
    
#     if ind_contol==0:
#       df_out["EAD"]= [rwa_raw_data["EAD"].sum()]
#       df_out["SA RWA"]= [rwa_raw_data["RWA_standardized"].sum()]  
#       # df_out["In Product RWA"]= [rwa_raw_data["rwa_airb"].sum()]  --gen 2 
#       df_out["Pre MOC RWA"]= [rwa_raw_data["RWA_new"].sum()]
      
#       df_out["Pre MOC RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
#       df_out["EAD (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["EAD"].sum()]

      
#       rwa_airb_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_Insured_class', 'RWA_standardized': 'RWA_standardized_by_Insured_class'}, axis='columns')
      
#       rwa_mapped_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_mapped_by_Insured_class'}, axis='columns')
      
#       rwa_airb_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_MRS_Bin'}, axis='columns')
      
#       rwa_mapped_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_mapped_by_MRS_Bin','RWA_standardized': 'RWA_standardized_by_MRS_Bin'}, axis='columns')
      
#       rwa_by_Insured_class0 =pd.merge(rwa_airb_by_Insured_class, rwa_mapped_by_Insured_class, how='left',on=['Insured_class'] )
      
#       rwa_by_MRS_Bin0 =pd.merge(rwa_airb_by_MRS_Bin, rwa_mapped_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )
      
#     if ind_contol==1: 
#         df_out["Post MOC Pre LR RWA"]= [rwa_raw_data["RWA_new"].sum()]
#         df_out["Post MOC Pre LR RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
    
#     '''   
#     if ind_contol==3: 
#         df_out["Post PD longrun"]= [rwa_raw_data["RWA_new"].sum()]
#         df_out["Post PD longrun (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]    
#     ''' 
        
#     if ind_contol==2: 
#         df_out["Final RWA"]= [rwa_raw_data["RWA_new"].sum()]

#         rwa_Post_MOC_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_Insured_class'}, axis='columns')

#         rwa_Post_MOC_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD'],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_MRS_Bin'}, axis='columns')

#         rwa_by_Insured_class =pd.merge(rwa_by_Insured_class0, rwa_Post_MOC_by_Insured_class, how='left',on=['Insured_class'] )

#         rwa_by_MRS_Bin =pd.merge(rwa_by_MRS_Bin0, rwa_Post_MOC_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )

#         #copy to res_data
#         res_data = rwa_raw_data.copy()
#         '''
#         with pd.ExcelWriter(
#           "C:\\Users\\jasonz\\OneDrive - Equitable Bank\\Fit_for_Use Development\\RWA\\eqb_rwa_original.xlsx",
#               mode="a",
#               engine="openpyxl",
#               if_sheet_exists="replace",
#           ) as writer:
#             res_data.to_excel(writer, sheet_name="RWA_Data") 
#         '''


In [148]:
# Save Data
with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_original.xlsx' )as writer:
    res_data.to_excel(writer, sheet_name='RWA_Data', index=False)
    df_out.to_excel(writer, sheet_name="RWA") 
    rwa_by_Insured_class.to_excel(writer, sheet_name="rwa_by_Insured_class")
    rwa_by_MRS_Bin.to_excel(writer, sheet_name="rwa_by_MRS_Bin")